In [1]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 6.9 MB/s 
     |████████████████████████████████| 453 kB 68.2 MB/s 


# 1.데이터 불러오기

In [209]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [210]:
import pandas as pd

In [211]:
df=pd.read_csv('/content/drive/MyDrive/total_review (2).csv',index_col=0)

In [212]:
from konlpy.tag import Mecab
from collections import Counter

In [213]:
import pandas as pd
import numpy as np  
import glob  
import sys
import os
import xlrd
import warnings
import re
warnings.filterwarnings('ignore')

In [214]:
import tweepy

# 2.자연어처리
   2-1. 형태소 분석(Okt), 코사인 유사도  
   2-2. 단어 문서 행렬(DTM)  
   2-3. 감성 분석

In [215]:
data_1=df.dropna(axis=0)
data_1.shape

(11494, 4)

In [216]:
data_1.head()

,name,rating,title,review
0,우도,1,쓰레기가 너무 많아요,쓰레기가 너무 많아요 바위 절벽과 바다는 절경이나... 섬 전체가 거대한 쓰레기장 ...
1,우도,3,너무 복잡해져버린 우도,너무 복잡해져버린 우도 예전에 갔던 기억으로 우도를 오랜만에 방문했는데 차도 많아졌...
2,우도,5,너무 예뻤던 인생바다,"너무 예뻤던 인생바다 날이 좋은 날 가니 바다빛이 동남아,하와이 휴양지 보다 더 예..."
3,우도,3,특별한 볼거리는 없는듯,특별한 볼거리는 없는듯 3만원 주고 전기차 시간 무제한...이라고 하지만 마지막 배...
4,우도,3,한번만 가볼만하다.,한번만 가볼만하다. 주머니 사정이 넉넉하지 않다면 순환 버스를 6000원에 이용하면...


##2-1. 형태소 분석, 코사인 유사도

In [9]:
from konlpy.tag import Okt
from collections import Counter

okt=Okt()
ii=[]
data_1=df.dropna(axis=0)
PP=[]
AA=[]
BB=[]
CC=[]
DD=[]
EE=[]
FF=[]
PP_1=[]

#명칭, 평점, 제목, 리뷰 데이터프레임화
for i in range(len(data_1)):
  AA.append(data_1.iloc[i,0]) #name
  BB.append(data_1.iloc[i,1]) #rating
  CC.append(data_1.iloc[i,2]) #title
  PP.append(data_1.iloc[i,3]) #review
  PP_1.append(data_1.iloc[i,3]) #raw


#리뷰 형태소 분석(어간 추출)
for i in range(len(PP)):
  PP[i]=re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ", PP[i])
  PP[i]=okt.morphs(PP[i],norm=True, stem=True)
  PP[i]=[x for x in PP[i] if len(x)>1]
  PP[i]=' '.join(PP[i])

data_2=pd.DataFrame({'name':AA,'rating':BB,'title':CC,'review':PP,'raw':PP_1,})


In [217]:
data_2.shape

(11494, 5)

In [11]:
hangul = re.compile('[ㄱ-ㅣ가-힣]+')

#리뷰를 구성한 단어가 1개일 시 제거
for i in range(len(data_2)):
  if len(hangul.findall(data_2.iloc[i,3]))>1:
    if len(data_2.iloc[i,3])>1:
      ii.append(i)

data_3=data_2.iloc[ii,:]

In [218]:
data_3.shape

(11443, 6)

In [13]:
a = data_3.review.to_list()
b = []
for i in range(len(a)):
  b += a[i].split(' ')


In [14]:
#총 단어 갯수(중복 포함)
len(b)

379489

In [15]:
#총 단어
words_count = {}
for word in b:
  if word in words_count: #dict에 단어가 있으면 1추가
    words_count[word] += 1
  else: #단어가 dict에 없으면 count에 단어 추가
    words_count[word] = 1
sorted_words = sorted([(k,v) for k,v in words_count.items()], key = lambda words_count: -words_count[1])
print([w[0]for w in sorted_words])


['하다', '있다', '좋다', '이다', '에서', '보다', '많다', '가다', '않다', '으로', '시간', '아름답다', '제주도', '같다', '너무', '방문', '없다', '되다', '제주', '정말', '사람', '사진', '바다', '아이', '해변', '폭포', '생각', '오다', '우리', '멋지다', '추천', '들다', '가보다', '에는', '여행', '많이', '다른', '되어다', '자다', '산책', '하지만', '아니다', '정도', '올라가다', '하고', '좋아하다', '박물관', '아주', '크다', '예쁘다', '까지', '장소', '한번', '매우', '우도', '공원', '코스', '싶다', '즐기다', '갈다', '자연', '가족', '때문', '걸다', '풍경', '함께', '먹다', '녹차', '입장료', '조금', '찍다', '경우', '구경', '작다', '근처', '최고', '동굴', '가장', '그냥', '날씨', '버스', '전망', '가면', '하나', '가지', '카페', '바람', '주변', '경치', '모든', '거리', '느끼다', '보이다', '힘들다', '보고', '다양하다', '느낌', '타고', '쉬다', '그렇다', '여기', '이나', '관광객', '그리고', '가치', '바위', '들어가다', '정원', '다시', '에도', '경험', '즐겁다', '따르다', '나오다', '비싸다', '유명하다', '이라', '특히', '오르다', '한라산', '체험', '여름', '가격', '바로', '찾다', '넓다', '관광지', '에게', '시장', '시원하다', '보기', '괜찮다', '나무', '가기', '또한', '관광', '찍기', '이쁘다', '정상', '겨울', '다음', '재미있다', '한국', '위치', '맛있다', '모습', '아이스크림', '흥미롭다', '계단', '입구', '걸리다', '차다', '여러', '별로', '보내다', '공항', '타다', '아쉽다', '

In [219]:
#많이 쓰인 단어 top10
sorted_words[:10]

[('하다', 18902),
 ('있다', 14691),
 ('좋다', 9201),
 ('이다', 5410),
 ('에서', 4419),
 ('보다', 4027),
 ('많다', 3595),
 ('가다', 3341),
 ('않다', 3289),
 ('으로', 2706)]

In [17]:
data_3.iloc[2,3]

'너무 예쁘다 인생 바다 좋다 가다 바다 동남아 하와이 휴양지 보다 예쁘다 하루 머무르다 여유 있다 둘러보다 싶다 생각 들다 이다 전기차 좌석 앉다 자다 보이지 않다 바닥 돌이 느껴지다 차감 돌아다니다 편하다 한데 타고 있다 너무 불편하다 특이하다 경험 이다'

### 2-2. Vector화

In [18]:
X = [[]]*len(data_3)
for i in range(len(data_3)):
  X[i] = data_3.iloc[i,3].split(' ') #review 단어 별로 나누기

#vector화
from gensim.models import Word2Vec
model_w = Word2Vec(sentences = X, size =500, window = 5, min_count=5, workers=4, sg=1)

In [19]:
#코사인 유사도 기준 카테고리 별 단어 리스트 생성(20개씩)
df1 = pd.DataFrame(model_w.wv.most_similar('멋있다', topn=20),columns = ['단어', '멋있다'])
df2 = pd.DataFrame(model_w.wv.most_similar('즐겁다',topn=20),columns = ['단어', '줄거운'])
df3 = pd.DataFrame(model_w.wv.most_similar('맛있다',topn=20),columns = ['단어', '맛있는'])
df4 = pd.DataFrame(model_w.wv.most_similar('독특하다',topn=20),columns = ['단어', '독특한'])
df5 = pd.DataFrame(model_w.wv.most_similar('쾌적하다',topn=20),columns = ['단어', '쾌적한'])
df6 = pd.DataFrame(model_w.wv.most_similar('편안하다',topn=20),columns = ['단어', '편안한'])
df7 = pd.DataFrame(model_w.wv.most_similar('친절하다',topn=20),columns = ['단어', '친절한'])
df8 = pd.DataFrame(model_w.wv.most_similar('신비롭다',topn=20),columns = ['단어', '신비한'])

In [20]:
#단어 리스트 확인
df8.shape

(20, 2)

In [21]:
#set활용 중복 단어 제거
A = list(set(df1.단어.tolist()+df2.단어.tolist()+df3.단어.tolist()+
             df4.단어.tolist()+df5.단어.tolist()+df6.단어.tolist()+
             df7.단어.tolist()+df8.단어.tolist()))
#정리
A = sorted(A)
#단어리스트 생성
pd.DataFrame(A).to_csv('단어리스트.csv', index = False, encoding='utf-8-sig')

In [22]:
A = pd.read_csv('단어리스트.csv',sep= ',', encoding = 'utf-8-sig' )
A = A.iloc[:,0].tolist()
A = sorted(A)

B_멋있는 = [] 
B_즐거운 = []
B_맛있는 = []
B_독특한 = []
B_쾌적한 = []
B_편안한 = []
B_친절한 = []
B_신비로운 = []

#코사인 유사도 리스트
for i in range(len(A)):
  B_멋있는.append(model_w.wv.similarity('멋있다',A[i]))
  B_즐거운.append(model_w.wv.similarity('즐겁다',A[i]))
  B_맛있는.append(model_w.wv.similarity('맛있다',A[i]))
  B_독특한.append(model_w.wv.similarity('독특하다',A[i]))
  B_쾌적한.append(model_w.wv.similarity('쾌적하다',A[i]))
  B_편안한.append(model_w.wv.similarity('편안하다',A[i]))
  B_친절한.append(model_w.wv.similarity('친절하다',A[i]))
  B_신비로운.append(model_w.wv.similarity('신비롭다',A[i]))

#카테고리 별 단어 유사도 모음
B_total = pd.DataFrame({'멋있는':B_멋있는,'즐거운':B_즐거운,'맛있는':B_맛있는,'독특한':B_독특한,
                        '쾌적한':B_쾌적한, '편안한':B_편안한, '친절한':B_친절한,'신비로운':B_신비로운})


#단어 모음을 칼럼으로 설정
B_total = B_total.T
B_total.columns = [A]
#유사도가 0.8이하이면 0으로 설정
B_total[B_total<0.8]=0

B_total.to_csv('단어리스트_2.csv', encoding = 'utf-8-sig', index = False)


In [23]:
#결과확인
B_total.shape

(8, 156)

In [208]:
B_total.head(2)

,가르치다,가슴,가이드,갈대,개월,거북이,건축,고요한,곳핫곳,과의,...,해질녘,행복하다,형성,형태,호수,혼란,환경,휴게,휴식,희미하다
멋있는,0.0,0.802842,0.0,0.823238,0.0,0.0,0.0,0.0,0.0,0.0,...,0.805588,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
즐거운,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.844498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##2-2. 단어 문서 행렬(DTM)

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
#빈도 기반
cv = CountVectorizer()
for i in range(len(X)):
  globals()['val{}'.format(i)] = []
B=[]
for i in range(len(X)):
  for j in X[i]:
    if j in A:
      globals()['val{}'.format(i)].append(j)
      B.append(i)



In [27]:
len(B)

7941

In [28]:

C=[]
for j in range(len(X)):
  if j not in list(set(B)):
    C.append('')
  else:
    C.append(' '.join(globals()['val{}'.format(j)])) 


      
  DTM_array = cv.fit_transform(C).toarray()
  feature_names = cv.get_feature_names()
  DTM_DataFrame = pd.DataFrame(DTM_array, columns = feature_names)

In [29]:
#문서 단어 행렬
DTM_DataFrame.shape

(11443, 156)

###행렬곱

In [35]:
import numpy as np
#행렬곱
Result = B_total.to_numpy() @ DTM_DataFrame.T.to_numpy()
#리뷰별 카테고리 연관성 데이터셋
round(pd.DataFrame(Result),4).to_csv('Relev.csv', encoding = 'utf-8-sig')
#카테고리별 코사인 유사도
B_total.to_csv('Word2Vec.csv', encoding = 'utf-8-sig')
#단어 문서 행렬
DTM_DataFrame.to_csv('DTM.csv',encoding = 'utf-8-sig')

##2-3. 감성분석

In [37]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
lexicon = pd.read_csv('감성 단어 리스트.csv', sep=',',encoding='utf-8-sig')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
#감성 단어 사전을 통한 감성분석
X = [[]]*len(data_3)
for i in range(len(data_3)):
  X[i] = data_3.iloc[i,3].split(' ')

A = [0]*len(data_3)
B = [0]*len(data_3)
C = [0]*len(data_3)
for i in range(len(data_3)):
  for j in range(len(X[i])):
    if X[i][j] in lexicon.word.tolist():
      A[i] += lexicon.polarity.tolist()[lexicon.word.tolist().index(X[i][j])]
      B[i] += 1
for i in range(len(C)):
  if B[i] != 0:
    C[i] = round(A[i]/B[i])

data_3['감성점수'] = C
data_3.to_csv('Sent.csv', index=False)

In [220]:
data_3

,name,rating,title,review,raw,감성점수
0,우도,1,쓰레기가 너무 많아요,쓰레기 너무 많다 바위 절벽 바다 절경 이나 전체 거대하다 쓰레기 느낌 바닷가 곳곳...,쓰레기가 너무 많아요 바위 절벽과 바다는 절경이나... 섬 전체가 거대한 쓰레기장 ...,-1
1,우도,3,너무 복잡해져버린 우도,너무 복잡하다 버리다 우도 예전 가다 기억 으로 우도 오랜 방문 하다 차도 많아지다...,너무 복잡해져버린 우도 예전에 갔던 기억으로 우도를 오랜만에 방문했는데 차도 많아졌...,-1
2,우도,5,너무 예뻤던 인생바다,너무 예쁘다 인생 바다 좋다 가다 바다 동남아 하와이 휴양지 보다 예쁘다 하루 머무...,"너무 예뻤던 인생바다 날이 좋은 날 가니 바다빛이 동남아,하와이 휴양지 보다 더 예...",1
3,우도,3,특별한 볼거리는 없는듯,특별하다 볼거리 없다 만원 주다 전기차 시간 무제한 이라고 하지만 마지막 여서 까지...,특별한 볼거리는 없는듯 3만원 주고 전기차 시간 무제한...이라고 하지만 마지막 배...,0
4,우도,3,한번만 가볼만하다.,한번 가보다 만하 주머니 사정 넉넉하다 않다 순환 버스 이용 하다 넉넉하다 스쿠터 ...,한번만 가볼만하다. 주머니 사정이 넉넉하지 않다면 순환 버스를 6000원에 이용하면...,0
...,...,...,...,...,...,...
11489,초콜릿 랜드,1,볼게 많지 않아요.,많다 않다 가다 후회 하다 입장료 만원 제품 구입 하다 사용 하다 쿠폰 받다 슈퍼마...,"볼게 많지 않아요. 이 곳에 가는 것을 후회했습니다. 입장료는 5만원, 3, 000...",0
11490,초콜릿 랜드,1,가지 마세요.,가지 말다 가장 나쁘다 우리 한국 에서 방문 하다 초콜릿 생산 도입 작고 대부분 우...,가지 마세요. 가장 나쁜 곳 우리는 한국에서 방문했습니다. 초콜릿 생산 도입은 작고...,0
11491,초콜릿 랜드,1,여행자에게 경고,여행자 에게 경고 편향 평가 아니다 설립 주인 방문자 의견 조언 주다 기쁘다 이다 ...,여행자에게 경고 이것은 편향 평가가 아닙니다. 설립 주인이 방문자의 의견을 조언 해...,0
11492,초콜릿 랜드,1,여행자 함정,여행자 함정 어른 입장권 천원 짜다 쿠폰 선물 줄다 있다 장교 에게 하다 하다 번째...,여행자 함정 어른 입장권 6000 원에 3 천원짜리 쿠폰을 선물로 사 주실 수 있습...,-1


#3. 리뷰 → 명소


In [40]:
#리뷰 별 카테고리 연관성
relev = pd.read_csv('Relev.csv', encoding='utf-8-sig')

In [41]:
relev = relev.drop('Unnamed: 0',axis =1)

In [42]:
relev.shape

(8, 11443)

In [43]:
relev.isnull().sum()

0        0
1        0
2        0
3        0
4        0
        ..
11438    0
11439    0
11440    0
11441    0
11442    0
Length: 11443, dtype: int64

In [44]:
#카테고리 -> 칼럼
relev = relev.T

#리뷰 name 달아주기
list2 = []
for i in data_3['name']:
  list2.append(i)

relev['name'] = list2

In [45]:
#카테고리 네임
category = []
for i in B_total.T.columns:
    category.append(i)

In [46]:
category

['멋있는', '즐거운', '맛있는', '독특한', '쾌적한', '편안한', '친절한', '신비로운']

In [47]:
#명소 별 칼럼 평균
place = relev.groupby('name').mean()
#컬럼 네이밍
place.columns = category

In [48]:
place.shape

(176, 8)

In [49]:
#명소별 리뷰 수
count_review =  relev.groupby('name').count()[0]
len(count_review)

176

In [50]:
#저장
place.to_csv('model.csv',encoding = 'utf-8-sig')

In [172]:
#필요한 데이터 불러오기
data_4 = data_3[['name','rating','감성점수']]
#name 기준 평균
data_5 = data_4.groupby('name').mean()

In [173]:
#데이터 합치기 [카테고리 + rating + 감성점수]
place_cat = pd.merge(place, data_5 ,on='name', how = 'left')

In [174]:
#칼럼 생성(리뷰수)
place_cat['count'] = count_review

In [175]:
place_cat

,멋있는,즐거운,맛있는,독특한,쾌적한,편안한,친절한,신비로운,rating,감성점수,count
name,,,,,,,,,,,
9.81 파크,0.000000,0.405300,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,4.500000,0.250000,4
성이시돌 목장,0.081300,0.000000,0.266627,0.170123,0.00000,0.000000,0.000000,0.147223,4.166667,1.000000,30
제주도립미술관,0.000000,0.000000,0.000000,0.434140,0.09034,0.084830,0.082000,0.184500,4.400000,1.600000,10
가파도,0.027100,0.028150,0.113067,0.057087,0.00000,0.111907,0.026880,0.000000,4.766667,1.166667,30
감귤 박물관,0.000000,0.243550,0.084770,0.000000,0.00000,0.000000,0.094870,0.000000,4.100000,0.900000,10
...,...,...,...,...,...,...,...,...,...,...,...
혼인지,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,2.666667,0.333333,3
환상숲곶자왈공원,0.000000,0.000000,0.000000,0.429680,0.17031,0.000000,0.082000,0.523470,4.600000,0.800000,10
황우지,0.032260,0.016212,0.000000,0.155472,0.01802,0.048658,0.000000,0.138672,4.240000,0.860000,50


In [176]:
#필요없는 명소 제거
place_cat = place_cat.T.drop(['트립어드바이저에서 전하는 말씀','트립어드바이저에서 전하는 말씀 •'],axis = 1)

In [177]:
#돌리기
place_cat = place_cat.T

In [178]:
place_cat.shape

(174, 11)

In [179]:
#저장
place_cat.to_csv('model_edit.csv',encoding = 'utf-8-sig')

In [180]:
df_test = pd.read_csv('model_edit.csv')

In [181]:
#검색 오류 해결
df_test['name'] = df_test['name'].apply(lambda x: x.replace(' ',''))

In [182]:
df_test.set_index('name',inplace = True)

In [ ]:
df_test.head()

In [184]:
cat_list = df_test.columns

In [185]:
#list확인
cat_list[:8]

Index(['멋있는', '즐거운', '맛있는', '독특한', '쾌적한', '편안한', '친절한', '신비로운'], dtype='object')

In [186]:
df_edit = pd.DataFrame()
#점수 = (1.2 * 카테고리 연관도 * 감성점수) + (0.1 * 평점) + (0.0005 * 리뷰 수)
for i in cat_list[:8]:
    df_edit[f'{i}'] =  (1.2 * df_test[f'{i}'] * df_test['감성점수'])+(0.1 * df_test['rating'])+(0.0005*df_test['count'])

In [187]:
#확인
df_edit.sort_values('독특한',ascending = False)

,멋있는,즐거운,맛있는,독특한,쾌적한,편안한,친절한,신비로운
name,,,,,,,,
제주도립김창열미술관,0.451000,1.180540,0.451000,2.743480,1.264420,0.451000,0.451000,1.240390
협재굴-한림공원,0.400500,0.400500,0.400500,2.445300,0.400500,0.400500,0.400500,1.426860
검멀레,0.501500,0.501500,1.066833,1.636967,0.501500,0.501500,0.501500,1.693900
조안베어뮤지엄,0.419667,0.419667,0.660867,1.571000,0.660440,0.419667,0.638333,1.109133
세계조가비박물관,0.526096,0.460000,0.460000,1.361587,0.460000,0.525651,0.602366,0.981583
...,...,...,...,...,...,...,...,...
제주항일기념관,0.300500,0.300500,0.300500,0.300500,0.300500,0.300500,0.300500,0.300500
파크써던랜드,0.300500,0.300500,0.300500,0.300500,0.300500,0.300500,0.300500,0.300500
혼인지,0.268167,0.268167,0.268167,0.268167,0.268167,0.268167,0.268167,0.268167


In [188]:
df_edit['rating'] = df_test['rating']

In [189]:
df_edit.isnull().sum()

멋있는       0
즐거운       0
맛있는       0
독특한       0
쾌적한       0
편안한       0
친절한       0
신비로운      0
rating    0
dtype: int64

In [193]:
df_edit.reset_index(inplace = True)

In [ ]:
for i in df_edit['name']:
  print(i)

In [195]:
#협재굴-한림공원, 민오름-오라동, 검멀레
df_edit['name'] = df_edit['name'].apply(lambda x : x.replace('-한림공원',''))
df_edit['name'] = df_edit['name'].apply(lambda x : x.replace('-오라동',''))
df_edit['name'] = df_edit['name'].apply(lambda x : x.replace('검멀레', '검멀레해변'))

In [ ]:
for i in df_edit['name']:
  print(i)

In [197]:
df_edit.set_index('name',inplace = True)

In [ ]:
df_edit.isnull().sum()

In [201]:
df_edit['rating'] = df_edit['rating'].apply(lambda x : round(x,1))

In [ ]:
df_edit['rating']

In [203]:
df_edit.to_csv('model_edit.csv',encoding = 'utf-8-sig')